In [1]:
import random
import math

# Lista completa com 30 cidades
cidades = [
    (5, 10), (15, 25), (30, 5), (40, 20), (20, 40),
    (35, 35), (10, 30), (50, 45), (45, 10), (60, 30),
    (25, 15), (55, 20), (70, 10), (80, 25), (65, 40),
    (90, 30), (75, 50), (85, 15), (95, 35), (40, 50),
    (10, 5), (20, 25), (35, 10), (50, 15), (60, 5),
    (70, 20), (30, 50), (45, 25), (55, 35), (65, 15)
]

ponto_inicial = (30, 30)  # Ponto de partida e chegada

n_total_cidades = 30    # Total de cidades disponíveis
cidades_visitadas = 15  # Apenas 15 serão visitadas
n_caixeiros = 3
cidades_por_caixeiro = cidades_visitadas // n_caixeiros  # 5 cidades por caixeiro

In [2]:
def distancia(p1, p2):
    return math.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

class Individuo:
    def __init__(self, cromossomo=None):
        # O cromossomo é uma permutação dos índices de 0 a 29.
        if cromossomo is None:
            self.cromossomo = list(range(n_total_cidades))
            random.shuffle(self.cromossomo)
        else:
            self.cromossomo = cromossomo
        self._fitness = None

    def fitness(self):
        if self._fitness is None:
            distancia_total = 0
            # Apenas as primeiras 15 cidades definem as rotas
            visitadas = self.cromossomo[:cidades_visitadas]
            for i in range(n_caixeiros):
                # Cada caixeiro visita 5 cidades
                rota = visitadas[i * cidades_por_caixeiro:(i + 1) * cidades_por_caixeiro]
                ponto_atual = ponto_inicial
                #indice da rota
                for idx in rota:
                    distancia_total += distancia(ponto_atual, cidades[idx])
                    ponto_atual = cidades[idx]
                distancia_total += distancia(ponto_atual, ponto_inicial)
            self._fitness = 1 / (1 + distancia_total)
        return self._fitness

    def mutacao(self):
        novo_cromossomo = self.cromossomo[:]
        i, j = random.sample(range(n_total_cidades), 2)
        novo_cromossomo[i], novo_cromossomo[j] = novo_cromossomo[j], novo_cromossomo[i]
        return Individuo(novo_cromossomo)

    def crossover(self, outro):
        inicio = random.randint(0, n_total_cidades - 2)
        fim = random.randint(inicio + 1, n_total_cidades - 1)
        filho = [None] * n_total_cidades
        filho[inicio:fim + 1] = self.cromossomo[inicio:fim + 1]
        pos = (fim + 1) % n_total_cidades
        pos_outro = (fim + 1) % n_total_cidades
        while None in filho:
            gene = outro.cromossomo[pos_outro]
            if gene not in filho:
                filho[pos] = gene
                pos = (pos + 1) % n_total_cidades
            pos_outro = (pos_outro + 1) % n_total_cidades
        return Individuo(filho)

    def imprime(self):
        visitadas = self.cromossomo[:cidades_visitadas]
        saida = ""
        for i in range(n_caixeiros):
            rota = visitadas[i * cidades_por_caixeiro:(i + 1) * cidades_por_caixeiro]
            coords_rota = [ponto_inicial] + [cidades[idx] for idx in rota] + [ponto_inicial]
            saida += f"Caixeiro {i + 1}: {coords_rota}\n"
        saida += f"Fitness: {self.fitness():.5f}"
        return saida

In [3]:
class Populacao:
    def __init__(self, tamanho_populacao=10):
        self.tamanho_populacao = tamanho_populacao
        self.populacao = []
    
    def inicializacao(self):
        self.populacao = [Individuo() for _ in range(self.tamanho_populacao)]
    
    def mutacao(self):
        return [ind.mutacao() for ind in self.populacao]
    
    def crossover(self):
        nova_lista = []
        for _ in range(self.tamanho_populacao):
            pai1, pai2 = random.sample(self.populacao, 2)
            nova_lista.append(pai1.crossover(pai2))
        return nova_lista
    
    def selecao(self, populacao_mutada=[], populacao_crossover=[]):
        combinado = self.populacao + populacao_mutada + populacao_crossover
        combinado.sort(key=lambda ind: ind.fitness(), reverse=True)
        self.populacao = combinado[:self.tamanho_populacao]
    
    def top_fitness(self):
        return self.top_individuo().fitness()
    
    def top_individuo(self):
        return self.populacao[0]

In [4]:
class AlgoritmoGeneticoPopulacao:
    def __init__(self, populacao):
        self.populacao = populacao
        self.erro = float('inf')
        self.geracoes = 1

    def erro_final(self):
        return self.erro

    def qtd_geracoes(self):
        return self.geracoes

    def rodar(self, max_geracoes=1000, imprimir_em_geracoes=100, erro_min=0.01):
        print(f"Geração: {self.geracoes}, Erro: {round(self.erro, 3)}\n{self.populacao.top_individuo().imprime()}")
        while True:
            if self.geracoes >= max_geracoes or self.erro <= erro_min:
                print(f"Geração: {self.geracoes}, Erro: {self.erro}\n{self.populacao.top_individuo().imprime()}")
                break
            mutantes = self.populacao.mutacao()
            filhos = self.populacao.crossover()
            self.populacao.selecao(mutantes, filhos)
            fitness = self.populacao.top_fitness()
            if (1 - fitness) < self.erro:
                self.erro = 1 - fitness
            self.geracoes += 1
            if self.geracoes % imprimir_em_geracoes == 0:
                print(f"Geração: {self.geracoes}, Erro: {self.erro}\n{self.populacao.top_individuo().imprime()}")
        return self.populacao.top_individuo()

In [5]:
if __name__ == "__main__":
    pop = Populacao(tamanho_populacao=10)
    pop.inicializacao()
    ag = AlgoritmoGeneticoPopulacao(pop)
    melhor = ag.rodar(max_geracoes=500, imprimir_em_geracoes=50, erro_min=0.001)
    print("\nMelhor solução encontrada:")
    print(melhor.imprime())

Geração: 1, Erro: inf
Caixeiro 1: [(30, 30), (50, 45), (15, 25), (30, 50), (5, 10), (55, 20), (30, 30)]
Caixeiro 2: [(30, 30), (25, 15), (75, 50), (20, 40), (45, 10), (60, 30), (30, 30)]
Caixeiro 3: [(30, 30), (55, 35), (65, 15), (80, 25), (95, 35), (30, 5), (30, 30)]
Fitness: 0.00159
Geração: 50, Erro: 0.9963218954130965
Caixeiro 1: [(30, 30), (25, 15), (15, 25), (10, 30), (20, 40), (20, 25), (30, 30)]
Caixeiro 2: [(30, 30), (50, 15), (45, 10), (70, 20), (70, 10), (65, 15), (30, 30)]
Caixeiro 3: [(30, 30), (45, 25), (40, 20), (60, 30), (55, 35), (35, 35), (30, 30)]
Fitness: 0.00368
Geração: 100, Erro: 0.9961145368023879
Caixeiro 1: [(30, 30), (25, 15), (15, 25), (10, 30), (20, 40), (20, 25), (30, 30)]
Caixeiro 2: [(30, 30), (40, 20), (65, 15), (70, 20), (70, 10), (45, 10), (30, 30)]
Caixeiro 3: [(30, 30), (45, 25), (55, 20), (60, 30), (55, 35), (35, 35), (30, 30)]
Fitness: 0.00389
Geração: 150, Erro: 0.9958843582816184
Caixeiro 1: [(30, 30), (25, 15), (15, 25), (10, 30), (20, 40), (20